In [ ]:
import json
##query_vector_database_tool (RAG)
import os
from openai import OpenAI

open_router_key = os.getenv("OPENROUTER_API_KEY", "")
BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemini-3-flash-preview"
openrouter = OpenAI(api_key=open_router_key, base_url=BASE_URL)

def call_llm(MODEL, system_message, user_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    response = openrouter.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content



## vector DB utilities for DHRP documents
import chromadb
from sentence_transformers import SentenceTransformer
from pathlib import Path
DB_PATH = r"C:\Users\rauna\projects\My Projects\IPO_Checker\ChromaDB"
chroma_client = chromadb.PersistentClient(path=DB_PATH)
COLLECTION_NAME = 'dhrp_embeddings_collection'
EMBEDDING_MODEL = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(EMBEDDING_MODEL)
dhrp_doc_collection = chroma_client.get_collection(COLLECTION_NAME)



def get_relevant_docs(question, collection, top_k=5):
    print('reteriving sentence documents for company...')

    embedded_question = embedding_model.encode(question)
    print(f"Question Embedded..")

    results = dhrp_doc_collection.query(
            query_embeddings=[embedded_question],
            n_results=top_k
            )
        # display(f"Top {top_k} relevant documents for question '{question}':{results}\n")
    return results


def build_context_json(documents):
    print('building context in JSON...')

    docs = documents.get("documents", [[]])[0]      
    metadatas = documents.get("metadatas", [[]])[0]

    context_list = []

    for doc, meta in zip(docs, metadatas):
        entry = {
            "source": meta.get("source", "unknown_source.txt").replace(".txt", ""),
            "page": meta.get("page", "Unknown page"),
            "content": doc.strip()
        }
        context_list.append(entry)

    return json.dumps({"context_results": context_list}, indent=4, ensure_ascii=False)


In [55]:
demo_param = [
    {'1':'one'},
    {'2':'two'},
    {'3':'three'}
]

In [ ]:
## List of dict implementation --- discarded

# def rag_pipeline(parameters,collection):

    
#     answers = []
#     for questions in parameters :

#         for key, question in questions.items() :

#             intermediate_dict = {}
            
#             documents = get_relevant_docs(question, collection)
#             context_json = build_context_json(documents)
        

#             system_message = f'''You are a Senior IPO Investment Analyst and SEBI-registered Research Analyst equivalent. 
#                 Use Indian IPO Draft Red Herring Prospectus (DRHP) documents to answer the questions accurately.
#                 Summaralize and reference the relevant sections from the DRHP documents in your answers.
#                 Use the following context to answer the questions.\n\nContext:\n
#                 {context_json}'''
        
#             # print(f'system_message prepared.{system_message}')

#             answer = call_llm(MODEL,system_message,question)
        
#             print('llm answers')
            
#             intermediate_dict[key] = answer
#             answers.append(intermediate_dict)
    

#     return answers

In [111]:
# rag answer pipeline that returns dict

def rag_pipeline(parameters,collection):

    
    answers = {}
    for questions in parameters :

        for key, question in questions.items() :

            
            documents = get_relevant_docs(question, collection)
            context_json = build_context_json(documents)
        

            system_message = f'''You are a Senior IPO Investment Analyst and SEBI-registered Research Analyst equivalent. 
                Use Indian IPO Draft Red Herring Prospectus (DRHP) documents to answer the questions accurately.
                Summaralize and reference the relevant sections from the DRHP documents in your answers.
                Use the following context to answer the questions.\n\nContext:\n
                {context_json}'''
        
            # print(f'system_message prepared.{system_message}')

            answer = call_llm(MODEL,system_message,question)
        
            print('llm answers')
            
            answers[key] = answer
    

    return answers

In [ ]:
# list of dict rag output

# rag_pipeline(eval_questions,dhrp_doc_collection)

reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers


[{'Manufacturing Capacity Utilization': "Based on the Draft Red Herring Prospectus (DRHP) for **KSH International Limited**, here is the analysis of the company’s manufacturing capacity and utilization levels:\n\n### **1. Current Capacity and Production Volume**\nAs of the most recent reported periods, the company's production and capacity metrics are as follows:\n\n| Particulars | Three-month period (ended June 30, 2025) | Fiscal 2025 | Fiscal 2024 | Fiscal 2023 |\n| :--- | :---: | :---: | :---: | :---: |\n| **Production Capacity (MT)** | 29,045 | 29,045 | 28,436 | 25,265 |\n| **Sales Volume (MT)** | 6,114 | 23,324 | 21,495 | 17,645 |\n\n*(Source: DRHP, Pages 129, 360)*\n\n### **2. Estimated Utilization Levels**\nWhile the DRHP (Page 44) contains a placeholder for a specific utilization table, the operational KPIs provided on pages 129 and 360 allow for the following calculation of the **Capacity Utilization** (Sales volume as a % of total capacity):\n\n*   **Fiscal 2025:** Approximat

In [ ]:
#dict output rag
# rag_pipeline(eval_questions,dhrp_doc_collection)

reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers


{'Manufacturing Capacity Utilization': "Based on the Draft Red Herring Prospectus (DRHP) for KSH International, here is the analysis of the company's manufacturing capacity and utilization levels:\n\n### **1. Current Capacity and Production Levels**\nAs of the most recent reporting periods, the company's production capacity and sales volumes (which serve as a proxy for utilized capacity) are as follows:\n\n| Particulars | Three-month period ended June 30, 2025 | Fiscal 2025 | Fiscal 2024 | Fiscal 2023 |\n| :--- | :--- | :--- | :--- | :--- |\n| **Production Capacity (MT)** | 29,045* | 29,045 | 28,436 | 25,265 |\n| **Magnet Winding Wires Sales Volume (MT)** | 6,114 | 23,324 | 21,495 | 17,645 |\n\n*\\*Note: The DRHP (page 129) lists the capacity for the three-month period as 29,045 MT, which typically refers to the annualized installed capacity available during that period.*\n\n### **2. Analysis of Utilization and Economies of Scale**\n*   **Calculated Utilization (FY25):** For Fiscal 202

In [ ]:
## List of dict eval parameters -- discarded

# eval_parameters = [
#     {"Manufacturing Capacity Utilization": "Closer to 100% utilization (to witness economies of scale and reduce cost per unit)"},
#     {"Raw Material Source Reliability": "Analyze potential trade disruptions if raw material comes from sensitive locations (e.g., China)"},
#     {"Management Future Plans": "A clean and clear thought process on future plans and strategies is preferred"},

#     # {"GDP Growth Rate (Annual)": "Check the specific rate and comparative performance (e.g., India growing faster than China)"},
#     # {"Labor Cost": "Check relative costs (e.g., labor cost in India was much lower than China)"},
#     # {"Macro Indicators": "Review unemployment rate, CRR, balance of trade, and current account to GDP percentage"},
#     # {"Industry Growth Rate (CAGR)": "The overall industry should be performing well. Compare current growth rate to expected growth rate"},
#     # {"Expected Industry Growth Rate": "Prefer expected growth rate to be higher than the current growth rate (e.g., Indian specialty chemical industry expected at 10.8% vs 9.8% previously)"},

#     # {"Revenue, EBITDA, and Profit After Tax (PAT) Growth Rate": "Should ideally be higher than the industry growth rate (e.g., 24% company growth vs 14% industry growth)"},
#     # {"Debt to Equity Ratio": "A ratio up to 2:1 is generally considered okay"},
#     # {"Cash Flow from Operations (CFO)": "Must be positive and growing at a good rate"},

#     # {"P/E Ratio (Price to Earning Ratio)": "Compare against listed peers; caution if the P/E is significantly higher (Use P/S ratio if the company is in losses)"},
#     # {"Return on Net Worth (RONW)": "Compare against peers; higher RONW is preferable"},
#     # {"Comparable Transaction Multiple (CTM)": "IPO price should be justified if it is way higher than the price the company issued shares at recently"},
#     # {"Gray Market Premium (GMP)": "Provides guidance: IPO Price + GMP = Expected Listing Price (not guaranteed)"},

#     # {"Type of Issue": "Prefer a Fresh Issue over a major Offer For Sale (OFS)"},
#     # {"Use of Proceeds": "Prefer funds used for expansion purpose over general corporate purpose"},
#     # {"Litigation/Investigation": "Be cautious if serious agencies like CBI or SFIO are investigating the promoters"},
#     # {"Customer Concentration": "Avoid when a small number of clients account for a high percentage of revenue (e.g., top 10 customers = 90%)"},
# ]

In [ ]:
#dict eval parameters

eval_parameters = {
    "Manufacturing Capacity Utilization": "Closer to 100% utilization (to witness economies of scale and reduce cost per unit)",
    "Raw Material Source Reliability": "Analyze potential trade disruptions if raw material comes from sensitive locations (e.g., China)",
    "Management Future Plans": "A clean and clear thought process on future plans and strategies is preferred"}

    # {"GDP Growth Rate (Annual)": "Check the specific rate and comparative performance (e.g., India growing faster than China)"},
    # {"Labor Cost": "Check relative costs (e.g., labor cost in India was much lower than China)"},
    # {"Macro Indicators": "Review unemployment rate, CRR, balance of trade, and current account to GDP percentage"},
    # {"Industry Growth Rate (CAGR)": "The overall industry should be performing well. Compare current growth rate to expected growth rate"},
    # {"Expected Industry Growth Rate": "Prefer expected growth rate to be higher than the current growth rate (e.g., Indian specialty chemical industry expected at 10.8% vs 9.8% previously)"},

    # {"Revenue, EBITDA, and Profit After Tax (PAT) Growth Rate": "Should ideally be higher than the industry growth rate (e.g., 24% company growth vs 14% industry growth)"},
    # {"Debt to Equity Ratio": "A ratio up to 2:1 is generally considered okay"},
    # {"Cash Flow from Operations (CFO)": "Must be positive and growing at a good rate"},

    # {"P/E Ratio (Price to Earning Ratio)": "Compare against listed peers; caution if the P/E is significantly higher (Use P/S ratio if the company is in losses)"},
    # {"Return on Net Worth (RONW)": "Compare against peers; higher RONW is preferable"},
    # {"Comparable Transaction Multiple (CTM)": "IPO price should be justified if it is way higher than the price the company issued shares at recently"},
    # {"Gray Market Premium (GMP)": "Provides guidance: IPO Price + GMP = Expected Listing Price (not guaranteed)"},

    # {"Type of Issue": "Prefer a Fresh Issue over a major Offer For Sale (OFS)"},
    # {"Use of Proceeds": "Prefer funds used for expansion purpose over general corporate purpose"},
    # {"Litigation/Investigation": "Be cautious if serious agencies like CBI or SFIO are investigating the promoters"},
    # {"Customer Concentration": "Avoid when a small number of clients account for a high percentage of revenue (e.g., top 10 customers = 90%)"},

In [97]:
eval_questions = [
    {"Manufacturing Capacity Utilization": "What is the current utilization level of the company's manufacturing capacity (to witness economies of scale and reduce cost per unit)?"},

    {"Raw Material Source Reliability": "How reliable are the company's raw material sources, and are there potential trade disruptions due to sourcing from sensitive locations such as China?"},

    {"Management Future Plans": "What are the company management's clearly stated future plans and strategies?"},

    # {"GDP Growth Rate (Annual)": "What is the current GDP growth rate and how does it compare with other relevant economies (e.g., India vs China)?"},

    # {"Labor Cost": "What is the comparative labor cost for the regions in which the company operates (e.g., India vs China)?"},

    # {"Macro Indicators": "What do the key macro indicators—unemployment rate, CRR, balance of trade, and current account to GDP percentage—indicate about the economic environment?"},

    # {"Industry Growth Rate (CAGR)": "What is the current CAGR of the industry, and is the overall industry performing well compared to historical or expected growth?"},

    # {"Expected Industry Growth Rate": "What is the expected future industry growth rate, and is it higher than the current growth rate?"},

    # {"Revenue, EBITDA, and Profit After Tax (PAT) Growth Rate": "What are the company's Revenue, EBITDA, and PAT growth rates, and are they higher than the industry growth rate?"},

    # {"Debt to Equity Ratio": "What is the company's current debt-to-equity ratio, and is it within the generally acceptable threshold of up to 2:1?"},

    # {"Cash Flow from Operations (CFO)": "Is the company's cash flow from operations positive and growing at a healthy rate?"},

    # {"P/E Ratio (Price to Earning Ratio)": "How does the company's P/E ratio compare with listed peers, and is the valuation justified (or should P/S be used if the company is loss-making)?"},

    # {"Return on Net Worth (RONW)": "What is the company's Return on Net Worth, and how does it compare to peer companies?"},

    # {"Comparable Transaction Multiple (CTM)": "Is the IPO price justified when compared with recent share issuance prices based on Comparable Transaction Multiples?"},

    # {"Gray Market Premium (GMP)": "What is the current Gray Market Premium (GMP), and what expected listing price does it imply (while noting it is not guaranteed)?"},

    # {"Type of Issue": "Is the IPO structured as a Fresh Issue, an Offer For Sale (OFS), or a mix—and what is the proportion of each?"},

    # {"Use of Proceeds": "How does the company intend to use the IPO proceeds, and what portion is allocated toward expansion versus general corporate purposes?"},

    # {"Litigation/Investigation": "Are there any ongoing litigations or investigations involving the company or promoters, especially by serious agencies like CBI or SFIO?"},

    # {"Customer Concentration": "What is the level of customer concentration, and do a small number of clients contribute disproportionately to revenue (e.g., top 10 customers = 90%)?"}
]


In [92]:
for i in range(len(eval_questions)) :
    topics = list(eval_questions[i].keys())
    print(topics)

['Manufacturing Capacity Utilization']
['Raw Material Source Reliability']
['Management Future Plans']


In [ ]:
# def ipo_rag_answerer(parameters,vector_collection = dhrp_doc_collection):
#     results = []
#     results.append(rag_pipeline(parameters, vector_collection))
#     return results

In [75]:
print(und)

[{'Manufacturing Capacity Utilization': 'Based on the DRHP for KSH International, the current utilization level of the company\'s manufacturing capacity is as follows:\n\nFor the **three-month period ended June 30, 2025**, the company utilized approximately **84.20%** of its annualized production capacity.\n\n### **Detailed Capacity and Utilization Analysis**\nDerived from the "Operating KPIs" and financial summaries (Pages 129, 360, and 375), the utilization metrics are:\n\n| Period | Production Capacity (MT) | Sales Volume (MT)* | Implied Utilization |\n| :--- | :--- | :--- | :--- |\n| **Three months ended June 30, 2025** | 29,045 (Annualized) | 6,114 | **~84.20%** (on quarterly basis) |\n| **Fiscal 2025** | 29,045 | 23,324 | **80.30%** |\n| **Fiscal 2024** | 28,436 | 21,495 | **75.59%** |\n| **Fiscal 2023** | 25,265 | 17,645 | **69.84%** |\n\n*\\*Note: Magnet winding wires sales volume is used as the primary proxy for production output in the KPI tables.*\n\n### **Impact on Economie

In [95]:
print(und)

[[{'Manufacturing Capacity Utilization': "Based on the Draft Red Herring Prospectus (DRHP) for KSH International, here is the analysis of the company's production capacity and utilization levels:\n\n### **1. Current Capacity and Utilization (FY 2025 June Quarter)**\nAs of the three-month period ended **June 30, 2025**, the company's operational performance was as follows:\n*   **Production Capacity:** 29,045 MT (Metric Tonnes)\n*   **Magnet Winding Wires Sales Volume:** 6,114 MT\n*   **Implied Capacity Utilization:** For the three-month period, the sales volume relative to the annual capacity suggests an annualized utilization rate of approximately **84.20%**.\n\n### **2. Historical Capacity and Volume Trends**\nThe company has shown a steady increase in capacity and sales volume, which typically aids in achieving economies of scale:\n\n| Financial Year | Production Capacity (MT) | Sales Volume (MT) | Volume Growth (%) |\n| :--- | :--- | :--- | :--- |\n| **Fiscal 2025** | 29,045 | 23,3

In [ ]:
# IPO evaluator that takes list of dict as rag_pipeline input and eval_parameter input  -- discarded
# 
# def ipo_evaluator(company_questions, ideal_answers):

#     results = []

#     for i in range(len(company_questions)):

#         topics = list(company_questions[i].keys())
#         # questions = company_questions[i].values()

#         company_answer = ipo_rag_answerer([company_questions[i]])[0].values()

#         for topic, company_answer in zip(topics, company_answer):
#             # print(f"Topic: {topic}, Answer: {company_answer}")
#             results.append({topic: company_answer})


#         ideal_answer = ideal_answers[i]
#         print(ideal_answer)
#         results.append({"ideal_answer": ideal_answer})

#         ipo_expert_prompt = '''You are an expert IPO analyst.
#                         Evaluate the company answer against the ideal answer and provide a score from 1 to 10, where 10 indicates a perfect match.  
#                         Consider accuracy, completeness, relevance, clarity, and depth of analysis in your evaluation.
#                         Provide only score and nothing else. 
#                         '''
        
#         evaluation_prompt = f'''Evaluate the following company answer against the ideal answer.\n\n
#                             Topic: {topic}\n\n
#                             Company Answer: {company_answer}\n\n
#                             Ideal Answer: {ideal_answer}\n\n
#                             Understand the Company Answer and give the result a score from 1 to 10.
#                             Only provide the score as a number from 1 to 10.'''
        
#         evaluation = call_llm(MODEL, ipo_expert_prompt, evaluation_prompt)
#         results.append({"evaluation_score": evaluation})
#     return results

In [ ]:
# # IPO evaluator that takes list of dict as rag_pipeline input and eval_parameter input  -- discarded

# 
# def ipo_evaluator(company_questions, ideal_answers):
#     """
#     Evaluates RAG performance. 
#     company_questions: List of dicts e.g., [{"risk": "What are the risks?"}]
#     ideal_answers: List of strings/dicts corresponding to the questions
#     """
#     final_report = []

#     for i, question_set in enumerate(company_questions):
#         # 1. Get RAG Answer
#         rag_results = ipo_rag_answerer([question_set])[0] 
#         ideal_val = ideal_answers[i]

#         for topic, actual_answer in rag_results.items():

#             # 2. Prepare Evaluation Prompt
#             ipo_expert_prompt = '''You are an expert IPO analyst.
#                         Evaluate the company answer against the ideal answer and provide a score from 1 to 10, where 10 indicates a perfect match.  
#                         Consider accuracy, completeness, relevance, clarity, and depth of analysis in your evaluation.
#                         Provide only score and nothing else. 
#                         '''
            
#             evaluation_prompt = f"""
#             Topic: {topic}
#             Actual Answer: {actual_answer}
#             Ideal Answer: {ideal_val}
#             Score the Actual Answer against the Ideal Answer (1-10).
#             Only provide the score as a number from 1 to 10.
#             """
            
#             score_raw = call_llm(MODEL, ipo_expert_prompt, evaluation_prompt)
            
#             # 3. Store Results
#             final_report.append({
#                 "topic": topic,
#                 "rag_answer": actual_answer,
#                 "ideal_answer": ideal_val,
#                 "score": score_raw.strip()
#             })
            
#     return final_report


In [133]:
#dict implementation

def ipo_evaluator(company_questions, ideal_answers,vector_collection = dhrp_doc_collection):
    """
    Evaluates RAG performance. 
    company_questions: List of dicts e.g., [{"risk": "What are the risks?"}]
    ideal_answers: List of strings/dicts corresponding to the questions
    """
    final_report = []

    company_answers = rag_pipeline(company_questions,vector_collection)
    
    for key,value in ideal_answers.items():
        ideal_answer = ideal_answers[key]
        # print(ideal_answer)
        company_answer = company_answers[key]
        # print(company_answer)

        #  Preparing Evaluation Prompt
        ipo_expert_prompt = '''You are an expert IPO analyst.
                        Evaluate the company answer against the ideal answer and provide a score from 1 to 10, where 10 indicates a perfect match.  
                        Consider accuracy, completeness, relevance, clarity, and depth of analysis in your evaluation.
                        Provide only score and nothing else. 
                        '''
            
        evaluation_prompt = f"""
            Topic: {key}
            Actual Answer: {company_answer}
            Ideal Answer: {ideal_answer}
            Score the Actual Answer against the Ideal Answer (1-10).
            Only provide the score as a number from 1 to 10.
            """
        
        score_raw = call_llm(MODEL, ipo_expert_prompt, evaluation_prompt)

        # 3. Store Results
        final_report.append({
                "topic": key,
                "rag_answer": company_answer,
                "ideal_answer": ideal_answer,
                "score": score_raw.strip()
            })
            
    return final_report


In [ ]:
# ipo_evaluator(eval_questions, eval_parameters, dhrp_doc_collection)

[{'Manufacturing Capacity Utilization': "What is the current utilization level of the company's manufacturing capacity (to witness economies of scale and reduce cost per unit)?"}]
0
[{'Raw Material Source Reliability': "How reliable are the company's raw material sources, and are there potential trade disruptions due to sourcing from sensitive locations such as China?"}]
1
[{'Management Future Plans': "What are the company management's clearly stated future plans and strategies?"}]
2


[]

In [ ]:
# ipo_evaluator(eval_questions, eval_parameters, dhrp_doc_collection)

1
Manufacturing Capacity Utilization
What is the current utilization level of the company's manufacturing capacity (to witness economies of scale and reduce cost per unit)?
1
Raw Material Source Reliability
How reliable are the company's raw material sources, and are there potential trade disruptions due to sourcing from sensitive locations such as China?
1
Management Future Plans
What are the company management's clearly stated future plans and strategies?


[]

In [ ]:
# ipo_evaluator(eval_questions, eval_parameters, dhrp_doc_collection)

reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers


[{'topic': 'Manufacturing Capacity Utilization',
  'rag_answer': 'Based on the Draft Red Herring Prospectus (DRHP) for KSH International, here is the analysis of the company\'s manufacturing capacity utilization:\n\n### **1. Current Capacity and Utilization (FY2025 Annualized)**\nFor the three-month period ended **June 30, 2025**, the company reported the following operational metrics:\n*   **Production Capacity:** 29,045 MT\n*   **Sales Volume (Magnet Winding Wires):** 6,114 MT\n*   **Implied Utilization (Quarterly):** Approximately **21.05%** for the single quarter.\n\n### **2. Historical Capacity Utilization Trends**\nTo assess the trajectory of economies of scale, the historical production capacity and sales volumes are as follows:\n\n| Particulars | Fiscal 2025 | Fiscal 2024 | Fiscal 2023 |\n| :--- | :--- | :--- | :--- |\n| **Production Capacity (MT)** | 29,045 | 28,436 | 25,265 |\n| **Sales Volume (MT)** | 23,324 | 21,495 | 17,645 |\n| **Utilization %** | **80.30%** | **75.59%** 

In [135]:
def avg_score(evaluation_result):
    total_score = 0
    count = 0

    for item in evaluation_result:
        if "score" in item:
            try:
                score = int(item["score"])
                total_score += score
                count += 1
            except ValueError:
                print(f"Invalid score format: {item['evaluation_score']}")

    average = total_score / count if count > 0 else 0
    return average  

In [136]:

# --- Gradio UI Wrapper ---
def run_ui_eval(ipo_name):
    # This is a placeholder for how you'd trigger it in the UI
    # In a real scenario, you'd fetch 'eval_questions' based on the ipo_name
    results = ipo_evaluator(eval_questions, eval_parameters,dhrp_doc_collection)
    avg = avg_score(results)
    return {"Average Score": avg, "Details": results}



In [137]:
# Using gr.Interface instead of ChatInterface for data processing

import gradio as gr
demo = gr.Interface(
    fn=run_ui_eval,
    inputs=gr.Textbox(label="Enter IPO Name"),
    outputs=gr.JSON(),
    title="IPO DRHP RAG Evaluator"
)

# if __name__ == "__main__":
#     demo.launch()

In [139]:
demo.launch()


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.


reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers
reteriving sentence documents for company...
Question Embedded..
building context in JSON...
llm answers


In [ ]:
# def ipo_result(ipo_name,history):
#     evaluation_result = ipo_evaluator(eval_questions, eval_parameters)
#     average_score = avg_score(evaluation_result)
#     result_summary = {
#         "IPO Name": ipo_name,
#         "Average Evaluation Score": average_score,
#         "Detailed Evaluation": evaluation_result
#     }
#     return result_summary